In [ ]:
import torch
import gpytorch
from matplotlib import pyplot as plt
import math
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.style.use('classic')

torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
df = pd.read_csv("https://www.ndbc.noaa.gov/data/realtime2/46268.txt",delim_whitespace=True)
df.info()

df.drop(index=df.index[0],
        axis=0,
        inplace=True)


In [ ]:
df['WVHT'] = df['WVHT'].astype(float)


In [ ]:
df = df.reindex(index=df.index[::-1])

In [ ]:
df

In [ ]:
df.rename(columns={"#YY": "year",
                   "MM": "month",
                   "DD": "day",
                   "hh": "hour",
                   "mm": "minute"})



In [ ]:
df['WVHT']


This is markdown



In [ ]:
train_y = torch.tensor(df['WVHT'].values.astype(np.float32))
train_x = torch.linspace(0,1,train_y.shape[0])

In [ ]:
from gpytorch.kernels import RBFKernel, CosineKernel, ScaleKernel, PeriodicKernel
from custom_kernel import MinKernel, AR2Kernel

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, covar_module):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = covar_module

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
kernel_1 = (
        ScaleKernel(AR2Kernel()) +
        ScaleKernel(MinKernel()) +
        ScaleKernel(RBFKernel())
        )
kernel_2 = (
        ScaleKernel(AR2Kernel()) +
        ScaleKernel(MinKernel()) +
        ScaleKernel(RBFKernel()) +
        ScaleKernel(RBFKernel()*PeriodicKernel())
        )


kernels = [kernel_1, kernel_2]

In [ ]:
training_iter = 150




# Use the adam optimizer

# "Loss" for GPs - the marginal log likelihood



In [ ]:
models = []
likelihoods = []

for kernel in kernels:
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(train_x, train_y, likelihood, kernel)

    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
    optimizer.param_groups[0]['capturable'] = True
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f    noise: %.3f' % (
            i + 1, training_iter, loss.item(),
            model.likelihood.noise.item()
        ))
        optimizer.step()
    models.append(model)
    likelihoods.append(likelihood)


In [ ]:
from utils import get_BIC

for i in range(len(kernels)):
    print(get_BIC(models[i], likelihoods[i], train_y, train_x))


In [ ]:
model = models[0]
likelihood = likelihoods[0]

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = torch.linspace(0,1.5,1000)
    observed_pred = likelihood(model(test_x))

In [ ]:
# Initialize plot
f, ax = plt.subplots(1, 1, figsize=(10, 7))

# Get upper and lower confidence bounds
lower, upper = observed_pred.confidence_region()
# Shade between the lower and upper confidence bounds
ax.fill_between(test_x.detach().cpu().numpy(),
                lower.detach().cpu().numpy(),
                upper.detach().cpu().numpy(), alpha=0.3)
# Plot training data as black stars
ax.scatter(train_x.detach().cpu().numpy(), train_y.detach().cpu().numpy(), s=0.5)
# Plot predictive means as blue line
ax.plot(test_x.detach().cpu().numpy(), observed_pred.mean.detach().cpu().numpy(), 'blue')

ax.set_ylim([0, 1.5])
#ax.patch.set_facecolor('green')
#ax.patch.set_alpha(.1)
ax.legend(["95% Credible Intervals", "Observed Data", "Posterior Mean"])
